In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import pandas as pd

In [3]:
from transformers import AutoModelForCausalLM, LlamaTokenizer
import torch

# Load the LLaMA tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", device_map="auto", load_in_4bit=True)

# Set the model to evaluation mode
model.eval()



ModuleNotFoundError: No module named 'transformers'

In [13]:
def load_column_descriptions(csv_file):
    df = pd.read_csv(csv_file)
    return df

# Step 2: Generate a detailed description using the LLaMA model
def generate_table_description(column_descriptions, model):
    table_description = "The table contains the following columns:\n\n"

    for _, row in column_descriptions.iterrows():
        column_name = row['Field']
        description = row['Data element']
        table_description += f"- {column_name}: {description}\n"

    # Generate a detailed description using the LLaMA model
    prompt = f"Given the following table structure, provide a short but complete description of the table:\n\n{table_description}\n\nDescription:"
    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    # Generate a response from the model
    with torch.no_grad():
        outputs = model.generate(**model_inputs)

    # Decode the generated text
    desc = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return desc

In [14]:
column_descriptions = load_column_descriptions("/content/DD02L_fields.csv")
detailed_description = generate_table_description(column_descriptions, model)

In [15]:
print(detailed_description)

Given the following table structure, provide a short but complete description of the table:

The table contains the following columns:

- TABNAME: Table Name
- AS4LOCAL: Activation Status of a Repository Object
- AS4VERS: Version of the entry (not used)
- TABCLASS: Table category
- SQLTAB: Name of an SQL table or an appended table
- DATMIN: Minimum no. of entries
- DATMAX: Maximum no. of entries
- DATAVG: Average number of entries
- CLIDEP: Flag indicating client-specific entries
- BUFFERED: Buffering flag
- COMPRFLAG: Field compression indicator
- LANGDEP: Language dependency
- ACTFLAG: Activation flag
- APPLCLASS: Application class for DD objects (not used)
- AUTHCLASS: Activation type
- AS4USER: Last Changed by
- AS4DATE: Date of Last Change
- AS4TIME: Last changed at
- MASTERLANG: Original Language in Repository objects
- MAINFLAG: Flag if Maintenance with Standard Tools is allowed
- CONTFLAG: Delivery class
- RESERVETAB: SDIC: Reserve for tables
- GLOBALFLAG: Flag for private DD o